# Code for Redacting Resumes Using Spacy 

# Redacted Dataset name = 'spacy_redacted_documents_with_id_and_category.csv'

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
#from docx import Document
import os
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pdfplumber
import nltk
nltk.download('punkt')
from textblob import TextBlob
import plotly.graph_objects as go
import plotly.express as px
import os
import json
#from docx import Document
import pdfplumber
from bs4 import BeautifulSoup
import spacy
import matplotlib.pyplot as plt
from wordcloud import WordCloud
#spacy.cli.download("en_core_web_sm")
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from collections import Counter
from plotly.offline import plot
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

## spacy

In [ ]:
resumes = pd.read_csv('Resumes_with_PII_updated.csv')

In [ ]:
#adjust this number to scale the size of the corpus
resumes = resumes.head(2000)

In [ ]:
resumes

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
def ner_pipeline(text):
    doc = nlp(text)
    entities = [{'word': ent.text, 'entity': ent.label_, 'score': ent.kb_id_} for ent in doc.ents]
    return entities

In [ ]:
structured_documents_spacy_corpus = []

In [ ]:
for index, row in resumes.iterrows():
    document_ID = row['ID']
    document_Category = row['Category']
    document_text = row['text']

    entities = ner_pipeline(document_text)

    document_data = {
        'ID': document_ID,
        'Category': document_Category,
        'text': document_text,
        'entities': entities  
    }

    structured_documents_spacy_corpus.append(document_data)

In [ ]:
for doc in structured_documents_spacy_corpus:
    print(doc)

In [ ]:
print(f"Total documents in structured_documents: {len(structured_documents_spacy_corpus)}")
for doc in structured_documents_spacy_corpus:
    print(f"ID: {doc['ID']}, Entities Count: {len(doc['entities'])}")


In [ ]:
spacy_corpus_redacted_text = []

In [ ]:
thresholds = {
    'PERSON': 0.75,
    'ORG': 0.99,
    'GPE': 0.99, 
}

In [ ]:
for i, document in enumerate(structured_documents_spacy_corpus):
    original_text = document['text']
    redacted_text = original_text
    entities_sorted = sorted(document.get('entities', []), key=lambda x: len(x['word']), reverse=True)

    print(f"Processing Document {i + 1}: {document['ID']}")
    print(f"Entities from document {i + 1}:")
    
    for entity in entities_sorted:
        entity_text = entity['word']
        entity_label = entity['entity']
        score = entity.get('score', 1.0)
        
       #Makes sure score is treated as a float
    score = entity.get('score')
    if isinstance(score, str):
        try:
            score = float(score)
        except ValueError:
            score = 0.0  #default if conversion fails
    else:
        score = score if score is not None else 1.0  #fallback to 1.0 if score no score
 
    
        if entity_label in thresholds and score >= thresholds[entity_label]:
            print(f"Entity: {entity['word']}, Label: {entity['entity']}, Score: {entity['score']:.4f}")

    for entity in entities_sorted:
        entity_text = entity['word']
        redacted_text = re.sub(re.escape(entity_text), '[Redacted]', redacted_text)

    redacted_text = re.sub(r'\S+@\S+', '[Redacted Email]', redacted_text)
    redacted_text = re.sub(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', '[Redacted Phone]', redacted_text)
    redacted_text = re.sub(r'\b(?:https?://)?(?:www\.)?linkedin\.com/in/[a-zA-Z0-9._-]+\b', '[Redacted Website]', redacted_text)
    
    
    spacy_corpus_redacted_text.append({
        'ID': document['ID'],  #use document instead of row be wary of rerunning code multiple times, dataset builds upon itsself
        'Category': document['Category'],    
        'Redacted Text': redacted_text  
    
    #spacy_corpus_redacted_text.append({
       # 'ID': row['ID'],                #row instead of document to build seperate columns in output
       # 'Category': row['Category'],    
       # 'Redacted Text': redacted_text   
    })

#list to a df
spacy_redacted_resumes = pd.DataFrame(spacy_corpus_redacted_text)

#Save to CSV
spacy_redacted_resumes.to_csv('spacy_redacted_documents_with_id_and_category.csv', index=False)


In [ ]:
print(f"Total number of redacted documents: {len(spacy_corpus_redacted_text)}")

In [ ]:
for i, text in enumerate(spacy_corpus_redacted_text):
    print(f"Redacted Text for Document {i + 1}:")
    print(text)

In [ ]:
document_index = 0

In [ ]:
if document_index < len(spacy_corpus_redacted_text):
    print(f"Redacted Text for Document 1:")
    print(spacy_corpus_redacted_text[document_index])
else:
    print("Document does not exist.")

In [ ]:
## spacy

resumes = pd.read_csv('Resumes_with_PII_updated.csv')

#adjust this number to scale the size of the corpus
resumes = resumes.head(2000)

resumes

nlp = spacy.load("en_core_web_lg")

def ner_pipeline(text):
    doc = nlp(text)
    entities = [{'word': ent.text, 'entity': ent.label_, 'score': ent.kb_id_} for ent in doc.ents]
    return entities

structured_documents_spacy_corpus = []

for index, row in resumes.iterrows():
    document_ID = row['ID']
    document_Category = row['Category']
    document_text = row['text']

    entities = ner_pipeline(document_text)

    document_data = {
        'ID': document_ID,
        'Category': document_Category,
        'text': document_text,
        'entities': entities  
    }

    structured_documents_spacy_corpus.append(document_data)

for doc in structured_documents_spacy_corpus:
    print(doc)

print(f"Total documents in structured_documents: {len(structured_documents_spacy_corpus)}")
for doc in structured_documents_spacy_corpus:
    print(f"ID: {doc['ID']}, Entities Count: {len(doc['entities'])}")


spacy_corpus_redacted_text = []

thresholds = {
    'PERSON': 0.75,
    'ORG': 0.99,
    'GPE': 0.99, 
}

for i, document in enumerate(structured_documents_spacy_corpus):
    original_text = document['text']
    redacted_text = original_text
    entities_sorted = sorted(document.get('entities', []), key=lambda x: len(x['word']), reverse=True)

    print(f"Processing Document {i + 1}: {document['ID']}")
    print(f"Entities from document {i + 1}:")
    
    for entity in entities_sorted:
        entity_text = entity['word']
        entity_label = entity['entity']
        score = entity.get('score', 1.0)
        
       #Makes sure score is treated as a float
    score = entity.get('score')
    if isinstance(score, str):
        try:
            score = float(score)
        except ValueError:
            score = 0.0  #default if conversion fails
    else:
        score = score if score is not None else 1.0  #fallback to 1.0 if score no score
 
    
        if entity_label in thresholds and score >= thresholds[entity_label]:
            print(f"Entity: {entity['word']}, Label: {entity['entity']}, Score: {entity['score']:.4f}")

    for entity in entities_sorted:
        entity_text = entity['word']
        redacted_text = re.sub(re.escape(entity_text), '[Redacted]', redacted_text)

    redacted_text = re.sub(r'\S+@\S+', '[Redacted Email]', redacted_text)
    redacted_text = re.sub(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', '[Redacted Phone]', redacted_text)
    redacted_text = re.sub(r'\b(?:https?://)?(?:www\.)?linkedin\.com/in/[a-zA-Z0-9._-]+\b', '[Redacted Website]', redacted_text)
    
    
    spacy_corpus_redacted_text.append({
        'ID': document['ID'],  #use document instead of row be wary of rerunning code multiple times, dataset builds upon itsself
        'Category': document['Category'],    
        'Redacted Text': redacted_text  
    
    #spacy_corpus_redacted_text.append({
       # 'ID': row['ID'],                #row instead of document to build seperate columns in output
       # 'Category': row['Category'],    
       # 'Redacted Text': redacted_text   
    })

#list to a df
spacy_redacted_resumes = pd.DataFrame(spacy_corpus_redacted_text)

#Save to CSV
spacy_redacted_resumes.to_csv('spacy_redacted_documents_with_id_and_category.csv', index=False)


print(f"Total number of redacted documents: {len(spacy_corpus_redacted_text)}")

for i, text in enumerate(spacy_corpus_redacted_text):
    print(f"Redacted Text for Document {i + 1}:")
    print(text)

document_index = 0

if document_index < len(spacy_corpus_redacted_text):
    print(f"Redacted Text for Document 1:")
    print(spacy_corpus_redacted_text[document_index])
else:
    print("Document does not exist.")